In [ ]:
from sim_helpers import *
from plotting_helpers import *

In [ ]:
num_economies = 50 # Number of nodes
m = 1 # Number of edges to attach from a new node to existing nodes

G = initialize_sim(num_economies, m)

In [ ]:
# Flow using paramters:
# x' = -k * x + m tanh(l * v) + SUM(i)[g * (x - x_i)]
# v' = -alpha * x + beta * v - c * v^3 + SUM(i)[g * (v - v_i)]

# define simulation time parameters
total_time = 300
t_divisions = 1000

# define master economy conditions
g = 0.5 #0.5 # base coupling strength
alpha = 0.4
beta = 0.6
c = 0.3
l = 0.3
k = 0.2
v = 0.2
m = 0.8

r0 = 1
g0 = 1


# run the simulation
data, times = run_simulation(G, total_time, t_divisions, x0=r0, v0=g0, g=g, param_perturb = 0.3,
                            alpha=alpha, beta=beta, c=c, l=l, k=k, v=v, m=m, ode_func=kaldor_model)

In [ ]:
kaldor_plt(G, data, times)

#### Calculating the order parameter for the Kaldor Business Model

In [ ]:
def kaldor_order_param(G, data, times):
    """
    Function will calculate the Order parameter (average) for the satellites in the kaldor network
    Arguments:
        G : The graph (networkx) with node having the name 'Master'
        data : The solution from ODE solver
        times : The array time stepped interval
    """
    # inital arrays and parameters
    N = int(np.shape(data)[1] / 2)
    r = np.zeros(shape = (np.shape(data)[0], N))
    g = np.zeros(shape = (np.shape(data)[0], N))

    # Separating Capital Stock and output into separate arrays
    for i in range(len(data)):
        ind = 0
        for j in range(N):
            r[i][j] = data[i][ind]
            g[i][j] = data[i][ind+1]
            ind+=2 

    r_avg = np.zeros(len(times))
    g_avg = np.zeros(len(times))

    # iterate over each time step
    for t in range(len(times)):
        r_sum = 0
        g_sum = 0
        # iterate over each node
        for i in G._node:
            # add to average only if not master
            if G._node[i]['name'] != 'Master':
                r_sum += r[t][i]
                g_sum += g[t][i]
        r_avg[t] += r_sum / (N-1)
        g_avg[t] += g_sum / (N-1)
    
    return r_avg, g_avg

r_avg, g_avg = kaldor_order_param(G, data, times)

#### Plotting the Order Parameter (average) and the Master economy.

Here we can see that as the coupling is increased, the order parameter amplitude increases (matching the Master economy). However, as the coupling is decreased, the order paramter amplitude decreases.

In [ ]:
# inital arrays and parameters
N = int(np.shape(data)[1] / 2)
r = np.zeros(shape = (np.shape(data)[0], N))
g = np.zeros(shape = (np.shape(data)[0], N))

# Separating Capital Stock and output into separate arrays
for i in range(len(data)):
    ind = 0
    for j in range(N):
        r[i][j] = data[i][ind]
        g[i][j] = data[i][ind+1]
        ind+=2 

# plotting capital stock v.s. time
plt.figure(figsize=(10,6))

c = 'orange'
ind = 0
for node in G._node:
    if G._node[node]['name'] == 'Master':
        p1, = plt.plot(times, r[:,ind], color = c)
    ind+=1
c = 'cornflowerblue'
p2, = plt.plot(times, r_avg, color = c)

plt.title('Capital Stock Space')
plt.xlabel('Time')
plt.ylabel('Capital Stock')
plt.xlim(0, times[-1])
plt.legend([p1, p2], ['Master Economy', 'Order Parameter (Average)'],
            handler_map={tuple: HandlerTuple(ndivide=None)})
plt.show()

# plotting output v.s. time
plt.figure(figsize=(10,6))

c = 'orange'
ind = 0
for node in G._node:
    if G._node[node]['name'] == 'Master':
        p1, = plt.plot(times, g[:,ind], color = c)
    ind+=1
c = 'cornflowerblue'
p2, = plt.plot(times, g_avg, color = c)

plt.title('Output Space')
plt.xlabel('Time')
plt.ylabel('Output')
plt.xlim(0, times[-1])
plt.legend([p1, p2], ['Master Economy', 'Order Parameter (Average)'],
            handler_map={tuple: HandlerTuple(ndivide=None)})
plt.show()

# plotting phase portrait
plt.figure(figsize=(10,6))

c = 'orange'
ind = 0
for node in G._node:
    if G._node[node]['name'] == 'Master':
        p1, = plt.plot(g[:,ind], r[:,ind], color = c)
    ind+=1
c = 'cornflowerblue'
p2, = plt.plot(g_avg, r_avg, color = c)

plt.title('Phase Space')
plt.xlabel('Output')
plt.ylabel('Capital Stock')
plt.legend([p1, p2], ['Master Economy', 'Order Parameter (Average)'],
            handler_map={tuple: HandlerTuple(ndivide=None)})
plt.show()



Modifying the run simulation function to have fixed paramters for testing the order parameter

In [ ]:
def define_simulation(G, total_time, t_divisions, x0=1, v0=1, g=1, param_perturb = 0.1,
                   mu=1, alpha=1, beta=1, c=1, l=1, k=1, v=1, m=1,ode_func = van_der_pol):
    """
    Function will define the simulation (calling the van_der_pol function - other can be specified)
    It takes your graph and adds random perturbation parameters to all of the nodes off of the 
    values of the master parameters (which are specified)

    Arguments:
        The arguments will depend on which model you are choosing and they will be used accordingly in the function call
        Although all parameters are added to the nodes, only certain ones may be used for different simulations.
        G : The graph (networkx) with 1 node having the name 'Master'
            If there is no node named 'Master' then the function will just couple all nodes normally
        total_time : The total time that the simulation should span
        t_divisions: The number of time step divisions that you want
        g : The coupling strength from the Master to other nodes (and base coupling from satellite nodes)
        param_perturb : a small perturbation order added (positive or negative) to each of the parameters (random)

        ###########
        van-der-pol
            Flow: (last term is the coupling to other nodes)
                x = v
                v = mu*(1 - x^2) * v - x - SUM(i)[g * (x - x_i)]
        x0 : The initial position of the master economy (base for satellite nodes)
        v0 : The initial velocity of the master economy (base for satellite nodes)
        mu : non-linearity and strength of damping (base for satellite nodes)

        ###########
        IS-LM model
            Flow: (last term is the coupling to other nodes)
                x' = beta * (k * v - v * x + m) + SUM(i)[g * (x - x_i)]
                v' = alpha * (c * v - mu * x + l) + SUM(i)[g * (v - v_i)]
        x0 : The initial Interest Rate of the master economy (base for satellite nodes)
        v0 : The initial Real Income (GDP) of the master economy (base for satellite nodes)
        mu : Suppressed investment in response to interest rate r (base for satellite nodes)
        l : Autonomous (government) expenditure (base for satellite nodes)
        c = p (1 − T ) + (i − 1) : Define full value when entering into model
            - p : Propensity to consume
            - T : Tax rate
            - i : Increased investment in response to income
        k : Enhanced demand for money in response to income y
        v : Suppressed demand for money in response to interest rate r
        alpha : positive rate for interest rate
        beta : positive rate for real income

        ###########
        Kaldor Business cycle model
            Flow: (last term is the coupling to other nodes)
                v' = -alpha * x + beta * v - c * v^3 + SUM(i)[g * (v - v_i)]
                x' = -k * x + m tanh(l * v) + SUM(i)[g * (x - x_i)]
        x0 : The initial Capital Stock of the master economy (base for satellite nodes)
        v0 : The initial Output of the master economy (base for satellite nodes)
        alpa : 
        beta : 
        c : 
        k : 
        m : 
        l : 

    """

    N = G.number_of_nodes() # number of nodes in the graph
    lnk = np.zeros(shape = (N,), dtype=int) # initialize an array to hold info on which nodes are linked
    ind = 0 # dummy index
    
    # this 1-D array will store the initial position and velocity of each node sequentially
    u0 = np.zeros(shape = (N * 2))
    
    # loop over each node and set the various parameters up
    for loop in G._node:
        G._node[loop]['index'] = ind
        G._node[loop]['link'] = list(nx.neighbors(G,loop))
        G._node[loop]['numlink'] = len(list(nx.neighbors(G,loop)))
        lnk[ind] = len(list(nx.neighbors(G,loop)))
        G._node[loop]['coupling'] = np.zeros(shape=(lnk[ind],))
        # for linkloop in range (lnk[ind]):
        #     G._node[loop]['coupling'][linkloop] = g + (random.random() - random.random()) * param_perturb
        #     while G._node[loop]['coupling'][linkloop] < 0: # check if this coupling is ok (otherwise this will cause a divergence)
        #         G._node[loop]['coupling'][linkloop] = g + (random.random() - random.random()) * param_perturb
        G._node[loop]['mu'] = mu + (random.random() - random.random()) * param_perturb
        G._node[loop]['alpha'] = alpha + (random.random() - random.random()) * param_perturb
        G._node[loop]['beta'] = beta + (random.random() - random.random()) * param_perturb
        G._node[loop]['c'] = c + (random.random() - random.random()) * param_perturb
        G._node[loop]['l'] = l + (random.random() - random.random()) * param_perturb
        G._node[loop]['k'] = k + (random.random() - random.random()) * param_perturb
        G._node[loop]['v'] = v + (random.random() - random.random()) * param_perturb
        G._node[loop]['m'] = m + (random.random() - random.random()) * param_perturb
        
        u0[ind*2 + 0] = random.random()
        u0[ind*2 + 1] = random.random()
        # can add additional factor of 10 if we are using ISLM_model
        if ode_func == van_der_pol:
            u0[ind*2 + 0]*=5
            u0[ind*2 + 1]*=5
        if ode_func == ISLM_model:
            u0[ind*2 + 0]*=50
            u0[ind*2 + 1]*=50
        

        # set the master initial conditions so we know what they are
        if G._node[loop]['name'] == 'Master':
            G._node[loop]['mu'] = mu
            G._node[loop]['alpha'] = alpha
            G._node[loop]['beta'] = beta
            G._node[loop]['c'] = c
            G._node[loop]['l'] = l
            G._node[loop]['k'] = k
            G._node[loop]['v'] = v
            G._node[loop]['m'] = m
            u0[ind*2 + 0] = x0
            u0[ind*2 + 1] = v0
            for linkloop in range (lnk[ind]):
                G._node[loop]['coupling'][linkloop] = g
        ind+=1

    return G, u0

def run_sim(G, total_time, t_divisions, u0, ode_func = van_der_pol):
    # set the time interval with time steps
    t = np.linspace(0, total_time, t_divisions)

    # run the ODE solver
    U = ode_func(G, u0, t)

    return U, t

In [ ]:
# Flow using paramters:
# x' = -k * x + m tanh(l * v) + SUM(i)[g * (x - x_i)]
# v' = -alpha * x + beta * v - c * v^3 + SUM(i)[g * (v - v_i)]

# define simulation time parameters
total_time = 300
t_divisions = 1000

# define master economy conditions
alpha = 0.4
beta = 0.6
c = 0.3
l = 0.3
k = 0.2
v = 0.2
m = 0.8

r0 = 1
g0 = 1

# inital arrays and parameters
N = int(np.shape(data)[1] / 2)
r = np.zeros(shape = (np.shape(data)[0], N))
g = np.zeros(shape = (np.shape(data)[0], N))

coupling = 0.01
iterations = 100
coupling_array = np.zeros(iterations) # will save the couplings in an array
r_amp = np.zeros(iterations) # will save the relative amplitude (squared) of r for each coupling g
g_amp = np.zeros(iterations) # will save the relative amplitude (squared) of g for each coupling g
r_max = np.zeros(iterations) # will save the relative amplitude (squared) of r for each coupling g
g_max = np.zeros(iterations) # will save the relative amplitude (squared) of g for each coupling g

# run the simulation
G, u0= define_simulation(G, total_time, t_divisions, x0=r0, v0=g0, g=coupling, param_perturb = 0.1,
                            alpha=alpha, beta=beta, c=c, l=l, k=k, v=v, m=m, ode_func=kaldor_model)

for i in range(iterations):
    coupling_array[i] = coupling
    
    N = G.number_of_nodes() # number of nodes in the graph
    lnk = np.zeros(shape = (N,), dtype=int) # initialize an array to hold info on which nodes are linked
    ind = 0 # dummy index
    for loop in G._node:
        G._node[loop]['index'] = ind
        G._node[loop]['link'] = list(nx.neighbors(G,loop))
        G._node[loop]['numlink'] = len(list(nx.neighbors(G,loop)))
        lnk[ind] = len(list(nx.neighbors(G,loop)))
        G._node[loop]['coupling'] = np.zeros(shape=(lnk[ind],))
        for linkloop in range (lnk[ind]):
            G._node[loop]['coupling'][linkloop] = coupling + (random.random() - random.random()) * 0.1
            while G._node[loop]['coupling'][linkloop] < 0: # check if this coupling is ok (otherwise this will cause a divergence)
                G._node[loop]['coupling'][linkloop] = coupling + (random.random() - random.random()) * 0.1
        # set the master initial conditions so we know what they are
        if G._node[loop]['name'] == 'Master':
            for linkloop in range (lnk[ind]):
                G._node[loop]['coupling'][linkloop] = coupling
        ind+=1

    data, times = run_sim(G, total_time, t_divisions, u0, ode_func = kaldor_model)

    # calculate the order parameters
    r_avg, g_avg = kaldor_order_param(G, data, times)
    r_amp[i] = np.average(r_avg**2)
    g_amp[i] = np.average(g_avg**2)
    
    # Separating Capital Stock and output into separate arrays
    for k in range(len(data)):
        ind = 0
        for j in range(N):
            r[k][j] = data[k][ind]
            g[k][j] = data[k][ind+1]
            ind+=2 

    ind = 0
    for node in G._node:
        # calculate the max only if its the master
        if G._node[node]['name'] == 'Master':
            g_max[i] = np.average(g[:,ind]**2)
            r_max[i] = np.average(r[:,ind]**2)
        ind+=1

    coupling += 0.01

In [ ]:
k_g = np.zeros(iterations)
k_r = np.zeros(iterations)

for i in range(iterations):
    k_g[i] = g_amp[i] / g_max[i]
    k_r[i] = r_amp[i] / r_max[i]

# plotting order parameters vs coupling strength
plt.figure(figsize=(10,6))

p1, = plt.plot(coupling_array, np.sqrt(k_r), color = 'cornflowerblue')
p2, = plt.plot(coupling_array, np.sqrt(k_g), color = 'orange')
plt.title('Noramlized Order Parameters vs Coupling')
plt.xlabel('Coupling')
plt.ylabel('Normalized Order Parameter Amplitude (Magnitude)')
plt.legend([p1, p2], ['Capital Stock Order Parameter', 'Output Order Parameter'],
            handler_map={tuple: HandlerTuple(ndivide=None)})
plt.show()